# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
   "com.snapchat.android_11.1.7.81-2109_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
   "com.snapchat.android_11.2.0.68-2110_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
#     "com.instagram.android_160.0.0.25.132-246889068_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_161.0.0.37.121-248310212_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_161.0.0.37.121-248310203_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_160.0.0.25.132-246889068_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_161.0.0.37.121-248310212_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.7vy',
    'COMMENT_LONGCLICK_HANDLER': 'X.3x7',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3tF',
    'DM_HOLDER': 'X.5B2',
    'DM_REPLY': 'X.46h',
    'DM_VOICEMESSAGE': 'X.4qE',
    'DMVIDEOPLAYER': 'X.7BE',
    'FEEDITEM': 'X.1V0',
    'FRAGMENT': 'X.1Pu',
    'HEADERGENERATOR': 'X.0Rv',
    'HTTPREQUESTTASKFACTORY': 'X.0pq',
    'INAPP_NOTIFICATION': 'X.6U7',
    'IGTV_VIEWER_FRAGMENT': 'X.7Ba',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.12t',
    'JACKSON_CREATOR': 'X.0iy',
    'LIVE_POST': 'X.1yR',
    'MAINFEED': 'X.1QT',
    'MEDIAITEM': 'X.1V2',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1mw',
    'NOTIFICATION_HANDLER': 'X.2WE',
    'PHOTOTIMERCONTROLLER': 'X.2s9',
    'POSTDETAIL': 'X.1di',
    'POSTGESTURE_SIMPLEPOST': 'X.1vx',
    'POSTGESTURE_CAROUSELIMAGE': 'X.26V',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2HS',
    'POSTVIEW': 'X.1dn',
    'PROFILE': 'X.0lO',
    'PROFILEDETAIL': 'X.33v',
    'PROFILEPIC_LONGCLICK': 'X.344',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2A4',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0nR',
    'NOTIFICATION_RUNNABLE': 'X.2eK',
    'SERIALSCHEDULER': 'X.0ip',
    'SESSION': 'X.0Mk',
    'SIMPLE_EXECUTOR': 'X.4Iy',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1Vh',
    'VIDEOSETTINGSMANAGER': 'X.1gs',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.7vy', '<init>', 'X.1z5', 'X.85d', 'X.0Mk', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.7vP', 'X.7w3'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3x7', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1H9', 'androidx.fragment.app.FragmentActivity', 'X.0Mk', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mk'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mk'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3tF', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.5B2', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.46h', 'A0L', 'X.46h', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.46h', 'A0I', 'X.46h'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.7BE', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1V0', 'A00', 'X.0jG'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.1Pu', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0Rv', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0pq', 'A03', 'java.lang.String', 'X.0Mk'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.6U7', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.7Ba', 'A00', 'X.7Ba'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.7Ba', 'A0C', 'X.7Ba'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.12t', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0iy', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0iy', '<init>', 'X.EWG'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.1yR', 'A00', 'X.0k7', 'X.1yS'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AKx'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1QT', 'configureActionBar', 'X.1Km'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1V2', 'A01', 'X.0jG', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0k7', 'X.1V2'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1mw', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2WE', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.2s9', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1di', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1vx', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.26V', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2HS', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1dn', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.33v', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0lO', 'A04', 'X.0Mk', 'X.0ju'),
    'PROFILE_ONLONGCLICK': MethodDec('X.344', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2A4', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BMT', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0j', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0Q', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0u', 'X.1p8', 'X.215'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0nR', 'A00', 'X.0nQ', 'X.11o'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.2eK', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0ip', 'A02', 'X.0qO'),
    'SESSION_GETTOKEN': MethodDec('X.0Mk', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4Iy', 'onFail', 'X.2EJ'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4Iy', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4Iy', '<init>', 'X.0Mk'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1Vh', 'A03', 'X.0k7', 'X.0jt'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1gs', 'A00', 'X.0Mk'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Kl', 'A09'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.1z5', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3x7', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1Wi', 'A0Y'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3tF', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.5BH', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.5BJ', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.4qE', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1V0', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0qN', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.7Ba', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1QT', 'A0P'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1Uh', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.215', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.215', 'A08'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.12Z', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1nn', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.6U7', 'A00'), FieldDec('X.2eK', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.6U7', 'A01'), FieldDec('X.2eK', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1td', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1tg', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.340', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.340', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.340', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.340', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.342', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.12Y', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.2sG', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1L'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1ln', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.215', 'A0D')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:
"""
c_decs = tuple()
m_decs = tuple()
f_decs = tuple()
"""

'\nc_decs = tuple()\nm_decs = tuple()\nf_decs = tuple()\n'

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1mw#A01() -> X.1oV#A01()
+ Found single candidate for Method. Considering it a match 
	X.0Rv#A00(android.content.Context) -> X.0SM#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback)
+ Found single candidate for M

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

~ Unique String not used in single class anymore. Change! (os_dark_mode_settings)
+ Found single candidate for Class. Considering it a match! 
	LX/0Mk; -> LX/0Mj;
+ Found single candidate for Class. Considering it a match! 
	LX/0ip; -> LX/0ir;
+ Found single candidate for Class. Considering it a match! 
	LX/0pq; -> LX/0qQ;
+ Found single candidate for Class. Considering it a match! 
	LX/0qN; -> LX/0uV;
+ Found single candidate for Class. Considering it a match! 
	LX/12Y; -> LX/1IC;
+ Found single candidate for Class. Considering it a match! 
	LX/12Z; -> LX/1ID;
+ Found single candidate for Class. Considering it a match! 
	LX/12t; -> LX/11A;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single ca

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/7vy; -> LX/6N3;
+ Found single candidate for Class. Considering it a match! 
	LX/3x7; -> LX/3Dz;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3tF; -> LX/37i;
+ Found single candidate for Class. Considering it a match! 
	LX/5B2; -> LX/4pi;
+ Found single candidate for Class. Considering it a match! 
	LX/7BE; -> LX/5LX;
+ Found single candidate for Class. Considering it a match! 
	LX/0iy; -> LX/0j0;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2s9; -> LX/2tp;
+ Found single candidate for Class. Considering it a match! 
	LX/1vx; -> LX/27t;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.26V#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2HS#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.344#A00(X.342, X.0Mk, X.0jt, android.content.Context, X.1Ih, X.2go, X.351)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.7vy#<init>(X.1z5, X.85d, X.0Mk, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.7vP, X.7w3) -> X.6N3#<init>(X.297, X.6Nj, X.0Mj, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.6Mg, X.6Iv)
+ Found single candidate for Method. Considering it a match 
	X.3x7#onLongPress(android.view.MotionEvent) -> X.3Dz#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLaunche

+ Found single candidate for Method. Considering it a match 
	X.0Mk#getToken() -> X.0Mj#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1Vh#A03(X.0k7, X.0jt) -> X.1Yp#A03(X.0k9, X.0jv)
Could resolve 0 new Classes, 12 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0jt; -> LX/0jv;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1Xb; -> LX/1bG;
+ Found single candidate for Class. Considering it a match! 
	LX/1yS; -> LX/2A5;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0qR; -> LX/0uh;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

71 / 145 Classes were matched
55 / 60 Methods were matched
35 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1mw -> X.1oV
• X.0Rv -> X.0SM
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.46h -> X.3nq
• X.0nR -> X.0nU
• X.1QT -> X.1Qs
• X.1gs -> X.1gi
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• X.2A4 -> X.2Ps
• X.33v -> X.2mC
• X.4Iy -> X.3zk
• X.6U7 -> X.6Fp
• X.2eK -> X.6Fq
• X.0Mk -> X.0Mj
• X.0ip -> X.0ir
• X.0pq -> X.0qQ
• X.0qN -> X.0uV
• X.12Y -> X.1IC
• X.12Z -> X.1ID
• X.12t -> X.11A
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Kl -> X.1Kj
• X.1Pu -> androidx.frag

• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) -> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• X.1Pu#getActivity() -> androidx.fragment.app.Fragment#getActivity()
• X.1yR#A00(X.0k7, X.1yS) -> X.2A4#A00(X.0k9, X.2A5)
• X.1V2#A01(X.0jG, boolean) -> X.1Vt#A01(X.0jI, boolean)
• com.instagram.feed.media.Media__JsonHelper#A00(X.0k7, X.1V2) -> com.instagram.feed.media.Media__JsonHelper#A00(X.0k9, X.1Vt)
• X.2WE#A01(android.content.Context, java.lang.CharSequence, int) -> X.2Sx#A01(android.content.Context, java.lang.CharSequence, int)
• X.1di#A05(X.1td, X.1V2, X.1uT, int, boolean, java.lang.String, X.0Mk, X.1Pz, java.lang.Integer, X.1hN, X.1Qi) -> X.1do#A05(X.1uR, X.1Vt, X.1vk, int, boolean, java.lang.String, X.0Mj, X.1QO, java.lang.Integer, X.1hF, X.1R7)
• X.0lO#A04(X.0Mk, X.0ju) -> X.0lS#A04(X.0Mj, X.0jw)
• com.instagram.reels.fragment.ReelViewerFragment#A0Q(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.6N3', '<init>', 'X.297', 'X.6Nj', 'X.0Mj', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.6Mg', 'X.6Iv'),
    MethodDec('X.3Dz', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1G5', 'androidx.fragment.app.FragmentActivity', 'X.0Mj', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagr

    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.11A'
    'X.0j0'
    'X.2A4'
    'X.1Qs'
    'X.1Vt'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1oV'
    'X.2Sx'
    'X.2tp'
    'X.1do'
    'X.27t'
    # No Match Found for Class 'X.26V'
    # No Match Found for Class 'X.2HS'
    'X.1dt'
    'X.0lS'
    'X.2mC'
    # No Match Found for Class 'X.344'
    'com.facebook.proxygen.JniHandler'
    'X.2Ps'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.0nU'
    'X.6Fq'
    'X.0ir'
    'X.0Mj'
    'X.3zk'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1Yp'
    'X.1gi'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1Kj', 'A09'),
    FieldDec('X.297', 'A00'),
    FieldDec('X.3Dz', 'A04'),
    # No Match Found for Field FieldDec('X.1Wi', 'A0Y'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewe

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [21]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk)
No matching Method found for X.26V#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2HS#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.344#A00(skip.params)
No matching Field found for X.1Wi#A0Y
No matching Field found for X.1td#A06
No matching Field found for X.1tg#A02
No matching Field found for X.342#A09


In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.6N3")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3Dz")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.37i")

    override val DM_HOLDER = ClassDec("X.4pi")

    override val DM_REPLY = ClassDec("X.3nq")

    override val DM_VOICEMESSAGE = ClassDec("X.4fD")

    override val DMVIDEOPLAYER = ClassDec("X.5LX")

    override val FEEDITEM = ClassDec("X.1Vr")

    override val FRAGMENT = ClassDec("androidx.fragment.app.F

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Kj"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.297"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3Dz"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1Wi"])
    override val COMMENT_TEXT = /* TODO */ VariableDec<String>("A0Y")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @